In [12]:
import gurobipy as gp
import numpy as np

In [13]:
inst="13"
sample="2"

core=np.loadtxt("Instancias/rcpsp/Datos30/core"+inst+sample+".txt", dtype='int')
pred=np.loadtxt("Instancias/rcpsp/Datos30/pred"+inst+sample+".txt", dtype='int')
dur=np.loadtxt("Instancias/rcpsp/Datos30/dura"+inst+sample+".txt", dtype='int')
recu=np.loadtxt("Instancias/rcpsp/Datos30/recu"+inst+sample+".txt", dtype='int')


In [14]:
J=[dur[i][0] for i in range(len(dur))]
n=len(J)
H=pred-1
d=[dur[i][1] for i in range(len(dur))]
K=recu[-1][0]
R=[recu[i][1] for i in range(len(recu))]
r=[[core[4*i+j][2] for j in range(K)] for i in range(n)]
es=[0 for i in range(n)]
Cmax=sum(d)
ls=[60 for i in range(n)]

In [116]:
P=np.zeros((n,n))
for h in range(len(H)):
    P[H[h,0],H[h,1]]=1

In [117]:
for j in range(n):
    es[j] = 0
    for i in range(j):
        if P[i,j] == 1:
            if es[j] < es[i] + d[i]:
                es[j] = es[i] + d[i]

for i in range(n-1,-1,-1):
    ls[i] = Cmax
    for j in range(i+1,n):
        if P[i,j] == 1:
            if ls[i] > ls[j] - d[i]:
                ls[i] = ls[j] - d[i]

In [118]:
RTC=gp.Model("Resource Constrained Project Scheduling Problem")

x=RTC.addVars(n,n, vtype=gp.GRB.BINARY, name='x')
y=RTC.addVars(n,n, vtype=gp.GRB.BINARY, name='y')
S=RTC.addVars(n, vtype=gp.GRB.CONTINUOUS, name='S')

In [119]:
RTC.setObjective(S[n-1], gp.GRB.MINIMIZE)

RTC.addConstrs((x[i,j]+x[j,i] <= 1  for i in range(n)
                                    for j in range(n)
                                    if i<j))

RTC.addConstrs((S[j]>= S[i] + d[i]-Cmax*(1-x[i,j])   for i in range(n)
                                                    for j in range(n)))

RTC.addConstrs((x[i,j] == 1 for i,j in H))

RTC.addConstr(S[0] == 0)

RTC.addConstrs(S[i] >= es[i] for i in range(n))
RTC.addConstrs((S[i] <= ls[i] for i in range(n)))

RTC.addConstrs((y[i,j]<=1-x[i,j]-x[j,i] for i in range(n)
                                        for j in range(n)))

RTC.addConstrs((S[i]<=S[j]+Cmax*(1-y[i,j])  for i in range(n)
                                            for j in range(n)))

RTC.addConstrs((S[i]+d[i]>=S[j]-Cmax*(1-y[i,j]) for i in range(n)
                                                for j in range(n)))

RTC.addConstrs((y[i,j]+y[j,i]+x[i,j]+x[j,i]>=1  for i in range(n)
                                                for j in range(n)
                                                if i<j))

RTC.addConstrs((r[i][k]+sum(r[j][k]*y[j,i] for j in range(n) if j!=i)<=R[k] for i in range(n)
                                                                            for k in range(K)))

RTC.update

<bound method Model.update of <gurobi.Model Continuous instance Resource Constrained Project Scheduling Problem: 0 constrs, 0 vars, No parameter changes>>

In [ ]:
RTC.setParam('OutputFlag',False)
RTC.setParam(gp.GRB.Param.TimeLimit,600)
RTC.optimize()
print(RTC.ObjVal)

In [ ]:
print("Objective = ",RTC.objVal)
print("Gap = ",RTC.MIPGap*100)
print("Running time = ",RTC.Runtime)
# for v in RTC.getVars():
#     if v.X > 0.1:
#         print('%s: %g' % (v.varName, v.x))
# print(Cmax)

In [5]:
opt=0
#gaps=[]
#objs=[]
for m in range(47,49):
    for n in range(1,11):
        inst=str(m)+str(n)

        core=np.loadtxt("Instancias/rcpsp/Datos30/core"+inst+".txt", dtype='int')
        pred=np.loadtxt("Instancias/rcpsp/Datos30/pred"+inst+".txt", dtype='int')
        dur=np.loadtxt("Instancias/rcpsp/Datos30/dura"+inst+".txt", dtype='int')
        recu=np.loadtxt("Instancias/rcpsp/Datos30/recu"+inst+".txt", dtype='int')

        J=[dur[i][0] for i in range(len(dur))]
        n=len(J)
        H=pred-1
        d=[dur[i][1] for i in range(len(dur))]
        K=recu[-1][0]
        R=[recu[i][1] for i in range(len(recu))]
        r=[[core[4*i+j][2] for j in range(K)] for i in range(n)]
        es=[0 for i in range(n)]
        ls=[sum(d) for i in range(n)]
        Cmax=sum(d)

        RTC=gp.Model("Resource Constrained Project Scheduling Problem")

        x=RTC.addVars(n,n, vtype=gp.GRB.BINARY, name='x')
        y=RTC.addVars(n,n, vtype=gp.GRB.BINARY, name='y')
        S=RTC.addVars(n, vtype=gp.GRB.CONTINUOUS, name='S')

        RTC.setObjective(S[n-1], gp.GRB.MINIMIZE)

        RTC.addConstrs((x[i,j]+x[j,i] <= 1  for i in range(n)
                                            for j in range(n)
                                            if i<j))

        RTC.addConstrs((S[j]>= S[i] + d[i]-Cmax*(1-x[i,j])   for i in range(n)
                                                            for j in range(n)))

        RTC.addConstrs((x[i,j] == 1 for i,j in H))

        RTC.addConstr(S[0] == 0)

        RTC.addConstrs((es[i] <= S[i] for i in range(n)))
        RTC.addConstrs((S[i] <= ls[i] for i in range(n)))

        RTC.addConstrs((y[i,j]<=1-x[i,j]-x[j,i] for i in range(n)
                                                for j in range(n)))

        RTC.addConstrs((S[i]<=S[j]+Cmax*(1-y[i,j])  for i in range(n)
                                                    for j in range(n)))

        RTC.addConstrs((S[i]+d[i]>=S[j]-Cmax*(1-y[i,j]) for i in range(n)
                                                        for j in range(n)))

        RTC.addConstrs((y[i,j]+y[j,i]+x[i,j]+x[j,i]>=1  for i in range(n)
                                                        for j in range(n)
                                                        if i<j))

        RTC.addConstrs((r[i][k]+sum(r[j][k]*y[j,i] for j in range(n) if j!=i)<=R[k] for i in range(n)
                                                                                    for k in range(K)))

        RTC.setParam('OutputFlag',False)
        RTC.optimize()
        if RTC.Status == gp.GRB.OPTIMAL:
            opt+=1
        print(RTC.ObjVal)
        #gaps=gaps+[RTC.MIPGap]
        #objs=objs+[RTC.ObjVal]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
58.0
59.0
55.0
48.0
44.99999900459898
50.99999999999998
63.0
48.0
65.0
57.0
63.0
54.0
50.0
57.0
58.0
58.0
55.0
44.0
59.0
54.0


In [1]:
total=48*10
print(total,opt)

NameError: name 'opt' is not defined